## Loading Modules

In [ ]:
import os
import numpy as np
import pandas as pd

from utils import *
from calibration import *

from synapticTrack.io import BeamDataIOManager
from synapticTrack.lattice.track_elements import *
from synapticTrack.lattice.track_lattice import Lattice
from synapticTrack.track.run_track import *

## Run Track Simulations

In [ ]:
import time
t0 = time.time()

# Load LEBT lattice
elements_list, stms = load_LEBT_lattice()

# ouput files and directories
output_dir_list = ['run.wire_scanner1', 'run.allison_scanner', 'run.wire_scanner2', 'run.wire_scanner3', 'run.wire_scanner4', 'run.ends']

final_output_files = ['coord_wire_scanner1.out', 'coord_allison_scanner.out', 'coord_wire_scanner2.out', 'coord_wire_scanner3.out', 
                      'coord_wire_scanner4.out', 'coord_ends.out']

fh = [5.0, 5.0, 5.0, 5.0, 5.0]
fv = [5.0, 5.0, 5.0, 5.0, 5.0]

sim_centroid_x = np.zeros((4, 5, 3), dtype=float)
sim_centroid_y = np.zeros((4, 5, 3), dtype=float)
count = 0

for i, stm in enumerate(['stm0', 'stm1', 'stm2', 'stm3', 'stm4']):
    for j, dir_end, kick in zip([0, 1, 2], ['p', 'm', 'n'], [1.1, -1.1, 0.0]):
        for k, direction in enumerate(['x', 'y']):
            fhkick = [0.0, 0.0, 0.0, 0.0, 0.0]
            fvkick = [0.0, 0.0, 0.0, 0.0, 0.0]
            if direction == 'x':
                fhkick[i] = kick
            elif direction == 'y':
                fvkick[i] = kick

            final_output_dir = "stm." + str(i) + direction + dir_end
            os.makedirs(final_output_dir, exist_ok=True)

            set_stm_strengths(stms, fhkick, fvkick)
            #elements_list = construct_lattice_segments()

            # run track
            run_track_steps(elements_list, output_dir_list)
            copy_step_output_files(final_output_dir, output_dir_list, final_output_files)
            copy_track_output_files(final_output_dir, output_dir_list)

            # compute beam centroids and rms sizes
            centroid, rms = get_beam_data(final_output_dir, final_output_files)
            x_center = centroid['x'].apply(lambda x: float(f"{x: 11.8f}")).tolist()
            y_center = centroid['y'].apply(lambda x: float(f"{x: 11.8f}")).tolist()

            for l, ws in enumerate([0, 2, 3, 4]):
                if direction == 'x':
                    sim_centroid_x[l, i, j] = x_center[ws]
                elif direction == 'y':
                    sim_centroid_y[l, i, j] = y_center[ws]
            
            print (f"stm{i}", f"{kick: 4.1f}", direction, x_center, y_center)

np.save("sim_centroid_x.npy", sim_centroid_x)
np.save("sim_centroid_y.npy", sim_centroid_y)

t1 = time.time()
print (t1 - t0)

## Calibrate Steering Magnet

In [ ]:
# ---------------------------------------------------------------
# Load numpy arrays (shape = (4 WS, 5 steerers, 3 kicks))
# kicks index order assumed to be: [ +1.1 mrad , -1.1 mrad , 0.0 ]
# ---------------------------------------------------------------
exp_x = np.load("exp_centroid_x.npy")   # shape (4,5,3)
exp_y = np.load("exp_centroid_y.npy")
sim_x = np.load("sim_centroid_x.npy")
sim_y = np.load("sim_centroid_y.npy")

print("Loaded centroid arrays with shape:", exp_x.shape)

WS = 4
STEERERS = 5
KICK_PLUS  = 0
KICK_MINUS = 1
KICK_ZERO  = 2

KICK_MAG = 1.1  # mrad nominal

In [ ]:
# ---------------------------------------------------------------
# Check data
# ---------------------------------------------------------------

check_data(WS, STEERERS, exp_x, sim_x, exp_y, sim_y)

In [ ]:
# ---------------------------------------------------------------
# Compute Wire Scanner Offsets
# ---------------------------------------------------------------

ws_offset_x, ws_offset_y = compute_wire_scanner_offsets(WS, exp_x, sim_x, exp_y, sim_y, KICK_ZERO)

np.save("ws_offset_x.npy", ws_offset_x)
np.save("ws_offset_y.npy", ws_offset_y)

In [ ]:
# ---------------------------------------------------------------
# Compute response matrices
# ---------------------------------------------------------------

from calibration import *
R_sim_x, R_exp_x = compute_response(WS, STEERERS, sim_x, exp_x, KICK_PLUS, KICK_MINUS, KICK_MAG)
R_sim_y, R_exp_y = compute_response(WS, STEERERS, sim_y, exp_y, KICK_PLUS, KICK_MINUS, KICK_MAG)

In [ ]:
# ---------------------------------------------------------------
# Compute scale factors
# ---------------------------------------------------------------

scale_x = solve_scale_factors(STEERERS, R_sim_x, R_exp_x)
scale_y = solve_scale_factors(STEERERS, R_sim_y, R_exp_y)

np.save("stm_scale_x.npy", scale_x)
np.save("stm_scale_y.npy", scale_y)

print_scale_factors(STEERERS, scale_x, scale_y)

In [ ]:
# ---------------------------------------------------------------
# Calibration summary
# ---------------------------------------------------------------
print_calibration_summary(WS, STEERERS, ws_offset_x, ws_offset_y, scale_x, scale_y)